In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.orm import Session,sessionmaker

#function(file,year)
#run the fucntion 10 times to get 10 df
#change column name,remove 1st row,add date cloumn, add year column
#combine 10 dataframes
#load final dataframe into database

In [2]:
#Declar the file path of each year
file_path_2019='raw_data/USHousing-2019.csv'
file_path_2018='raw_data/USHousing-2018.csv'
file_path_2017='raw_data/USHousing-2017.csv'
file_path_2016='raw_data/USHousing-2016.csv'
file_path_2015='raw_data/USHousing-2015.csv'
file_path_2014='raw_data/USHousing-2014.csv'
file_path_2013='raw_data/USHousing-2013.csv'
file_path_2012='raw_data/USHousing-2012.csv'
file_path_2011='raw_data/USHousing-2011.csv'
file_path_2010='raw_data/USHousing-2010.csv'

In [3]:
#function to transform the raw data
def data_transf(file_path,year):
    origin_df=pd.read_csv(file_path)
    
    extract_df=origin_df[['NAME','DP04_0001E','DP04_0002E','DP04_0002PE','DP04_0007E','DP04_0008E','DP04_0009E',
                       'DP04_0010E','DP04_0011E','DP04_0012E','DP04_0013E','DP04_0014E','DP04_0015E',
                       'DP04_0039E','DP04_0040E','DP04_0041E','DP04_0042E','DP04_0043E','DP04_0044E',
                       'DP04_0080E','DP04_0089E','DP04_0110E','DP04_0111E','DP04_0112E',
                       'DP04_0113E','DP04_0114E','DP04_0115E','DP04_0116E','DP04_0117E','DP04_0118E','DP04_0119E',
                       'DP04_0120E','DP04_0121E','DP04_0122E','DP04_0123E','DP04_0124E','DP04_0125E']]
    
    renamed_df=extract_df.rename(columns={'NAME':'state','DP04_0001E':'total_housing_units','DP04_0002E':'occupied_units','DP04_0002PE':'occupied_perc','DP04_0007E':'unit_det_1',
                                'DP04_0008E':'unit_att_1','DP04_0009E':'units_2','DP04_0010E':'units_3or4','DP04_0011E':'units_5to9','DP04_0012E':'units_10to19',
                                'DP04_0013E':'units_20more','DP04_0014E':'mobile_home','DP04_0015E':'boat_rv_van','DP04_0039E':'no_bedroom','DP04_0040E':'one_bedroom',
                                'DP04_0041E':'two_bedrooms','DP04_0042E':'three_bedrooms','DP04_0043E':'four_bedrooms','DP04_0044E':'five_ormore_bedrooms','DP04_0080E':'owner_occupied_units',
                                'DP04_0089E':'house_median_value','DP04_0110E':'units_smocapi_m','DP04_0111E':'smocapi_20_perc_less_m','DP04_0112E':'smocapi_20to249_perc_m',
                                'DP04_0113E':'smocapi_25to299_perc_m','DP04_0114E':'smocapi_30to349_perc_m','DP04_0115E':'smocapi_35perc_more_m','DP04_0116E':'smocapi_not_computed_m',
                                'DP04_0117E':'units_smocapi_nm','DP04_0118E':'smocapi_10perc_less_nm','DP04_0119E':'smocapi_10to149_perc_nm','DP04_0120E':'smocapi_15to199_perc_nm',
                                'DP04_0121E':'smocapi_20to249_perc_nm','DP04_0122E':'smocapi_25to299_perc_nm','DP04_0123E':'smocapi_30to349_perc_nm','DP04_0124E':'smocapi_35perc_more_nm',
                                'DP04_0125E':'smocapi_not_computed_nm'})
    
    renamed_df['year']=year
    renamed_df['date']=f'{year}-01-01'
    
    row_df=renamed_df.iloc[1:52,:]
    
    
    return row_df


In [4]:
#cleanup/transform each dataset
final_2019=data_transf(file_path_2019,2019)
final_2018=data_transf(file_path_2019,2018)
final_2017=data_transf(file_path_2019,2017)
final_2016=data_transf(file_path_2019,2016)
final_2015=data_transf(file_path_2019,2015)
final_2014=data_transf(file_path_2019,2014)
final_2013=data_transf(file_path_2019,2013)
final_2012=data_transf(file_path_2019,2012)
final_2011=data_transf(file_path_2019,2011)
final_2010=data_transf(file_path_2019,2010)

In [5]:
merged_df=pd.concat([final_2019,final_2018,final_2017,final_2016,final_2015,final_2014,final_2013,final_2012,final_2011,final_2010])
final_df=merged_df.reset_index(drop=True)

state_code_df=pd.read_csv("raw_data/states-code.csv")
state_codes=state_code_df.rename(columns={'State':'state','Abbrev':'abbrev','Code':'code'})

left_join=pd.merge(final_df,state_codes,on='state',how='left')
left_join['primary_key']=left_join['code']+'_'+left_join['year'].astype(str)

In [6]:
#left_join.to_csv('final_dataset.csv',index=False)

In [7]:
# create your class
class Housing(Base):
    __tablename__ = 'housing_10to19'
    __table_args__ = {'extend_existing': True} 
    state = Column(String)
    total_housing_units = Column(Integer)
    occupied_units = Column(Integer)
    occupied_perc = Column(Float)
    unit_det_1=Column(Integer)
    unit_att_1 = Column(Integer)
    units_2 = Column(Integer)
    units_3or4 = Column(Integer)
    units_5to9 = Column(Integer)
    units_10to19 = Column(Integer)
    units_20more = Column(Integer)
    mobile_home = Column(Integer)
    boat_rv_van = Column(Integer)
    no_bedroom = Column(Integer)
    one_bedroom = Column(Integer)
    two_bedrooms = Column(Integer)
    three_bedrooms = Column(Integer)
    four_bedrooms = Column(Integer)
    five_ormore_bedrooms = Column(Integer)
    owner_occupied_units = Column(Integer)
    house_median_value = Column(Integer)
    units_smocapi_m = Column(Integer)
    smocapi_20_perc_less_m = Column(Integer)
    smocapi_20to249_perc_m = Column(Integer)
    smocapi_25to299_perc_m = Column(Integer)
    smocapi_30to349_perc_m = Column(Integer)
    smocapi_35perc_more_m = Column(Integer)
    smocapi_not_computed_m = Column(Integer)
    units_smocapi_nm = Column(Integer)
    smocapi_10perc_less_nm = Column(Integer)
    smocapi_10to149_perc_nm = Column(Integer)
    smocapi_15to199_perc_nm = Column(Integer)
    smocapi_20to249_perc_nm = Column(Integer)
    smocapi_25to299_perc_nm = Column(Integer)
    smocapi_30to349_perc_nm = Column(Integer)
    smocapi_35perc_more_nm = Column(Integer)
    smocapi_not_computed_nm = Column(Integer)
    year = Column(Integer)
    date= Column(String)
    abbrev = Column(String)
    code= Column(String)
    primary_key = Column (String,primary_key=True)

In [8]:
database_path = "us_housing_db.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [9]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [10]:
#left_join.to_dict(orient="records")

In [11]:
Session = sessionmaker(bind=conn)
session = Session()
session.bulk_insert_mappings(Housing, left_join.to_dict(orient="records"))
session.commit()
session.close()

In [ ]:
#'DP04_0081E','DP04_0082E','DP04_0083E','DP04_0084E','DP04_0085E','DP04_0086E','DP04_0087E','DP04_0088E',

#'DP04_0081E':'$50k_less','DP04_0082E':'$50000to$99,999','DP04_0083E':'$100000to$149,999','DP04_0084E':'$150000to$199999',
#'DP04_0085E':'$200000to$299999','DP04_0086E':'$300000to$499999','DP04_0087E':'$500000to$999999','DP04_0088E':'$1M_more',